<a href="https://colab.research.google.com/github/poojavjpy/Locust-Detection/blob/main/YOLOv8_Augmented.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Mounting Google Drive in Colab***

In [ ]:
# Mount Google Drive to access your dataset
from google.colab import drive
drive.mount('/content/drive')


***Installing Ultralytics for YOLOv8***

In [ ]:
!pip install ultralytics


***Import necessary libraries***

In [ ]:
# Import necessary libraries
import os
from ultralytics import YOLO




***Clearing the cache***

In [ ]:
!rm /content/drive/MyDrive/locust_detection/dataset/roboflow/labels/train.cache

In [ ]:
!rm /content/drive/MyDrive/locust_detection/dataset/roboflow/labels/val.cache

***Dataset Loading***

In [ ]:
# Define the paths to your dataset and where to save the model
dataset_path = '/content/drive/MyDrive/locust_detection/dataset/augmented'  # Path to your YOLOv5 formatted dataset
save_model_path = '/content/drive/MyDrive/yolov8_model/robo yolo v8/augmented'  # Path to save the trained YOLOv8 model

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')  # 'yolov8n.pt' is the nano version of YOLOv8. You can use 'yolov8s.pt', 'yolov8m.pt', etc. for different sizes


***Training of the model***

In [ ]:
# Define the correct path to your dataset's data.yaml
dataset_yaml_path = '/content/drive/MyDrive/locust_detection/yolov5/datasett.yaml'  # Ensure this is the correct path

# Train the YOLOv8 model using the correct path
results = model.train(
    data=dataset_yaml_path,  # Use the correct path here
    epochs=60,
    batch=16,
    imgsz=640,
    save_period=10,
    project=save_model_path,
    name='yolov8_detection'
)


***Testing Single Image***

In [ ]:
from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt
import os
import shutil

# Define paths
weights_path = '/content/drive/MyDrive/yolov8_model/robo yolo v8/augmented/yolov8_detection2/weights/last.pt'  # Path to the trained model weights
image_path = '/content/drive/MyDrive/locust_detection/test_data/images/666.jpg'  # Path to the image for testing
output_directory = '/content/drive/MyDrive/yolov8_output/robo yolo v8'  # Directory to save output images

# Load the YOLOv8 model
model = YOLO(weights_path)

# Perform inference on the image
results = model.predict(source=image_path, conf=0.40, save=True, project=output_directory, name='detection_results')

# Print the results
print(f"Detection results for {image_path}:")
for result in results:
    boxes = result.boxes.xyxy  # Bounding box coordinates (xmin, ymin, xmax, ymax)
    confidences = result.boxes.conf  # Confidence scores
    class_ids = result.boxes.cls  # Class IDs

    for box, confidence, class_id in zip(boxes, confidences, class_ids):
        print(f"Class: {int(class_id)}, Confidence: {confidence:.2f}, Box: {box.tolist()}")

# Define the path to the saved output image
# YOLOv8 might create a subfolder; handle this by moving the image
# Check for subfolders and move images to the target directory
for root, dirs, files in os.walk(output_directory):
    for file in files:
        if file.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            src_path = os.path.join(root, file)
            dst_path = os.path.join(output_directory, file)
            shutil.move(src_path, dst_path)

# Define the path to the moved image
saved_image_path = os.path.join(output_directory, os.path.basename(image_path))

# Print out saved image directory to debug path issues
print(f"Expected path for saved image: {saved_image_path}")

# Load and display the saved image with bounding boxes
if os.path.exists(saved_image_path):
    img = Image.open(saved_image_path)

    # Display the image with bounding boxes
    plt.figure(figsize=(10, 10))
    plt.imshow(img)
    plt.axis('off')  # Hide axes
    plt.show()
else:
    print(f"Saved image not found at {saved_image_path}")


***Testing Multiple Images***

In [ ]:
from ultralytics import YOLO
import os
import shutil
from PIL import Image
import matplotlib.pyplot as plt

# Define paths
weights_path = '/content/drive/MyDrive/yolov8_model/robo yolo v8/augmented/yolov8_detection2/weights/last.pt'  # Path to the trained model weights
source_directory = '/content/drive/MyDrive/locust_detection/test_data/images'  # Directory containing images
output_directory = '/content/drive/MyDrive/yolov8_output/robo yolo v8/augmented3'  # Directory to save output images

# Load the YOLOv8 model
model = YOLO(weights_path)

# Perform inference on all images in the source directory
results = model.predict(source=source_directory, conf=0.40, save=True, project=output_directory, name='detection_results')

# Move images if YOLOv8 saves them in subfolders
for root, dirs, files in os.walk(output_directory):
    for file in files:
        if file.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            src_path = os.path.join(root, file)
            dst_path = os.path.join(output_directory, file)
            shutil.move(src_path, dst_path)

# Function to resize images
def resize_image(image_path, size=(640, 480)):
    with Image.open(image_path) as img:
        img = img.resize(size, Image.Resampling.LANCZOS)  # Updated from ANTIALIAS to LANCZOS
        return img

# Display resized images
image_files = [f for f in os.listdir(output_directory) if f.endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
num_images = len(image_files)

# Set grid size
cols = 5
rows = (num_images // cols) + (num_images % cols > 0)  # Number of rows needed

plt.figure(figsize=(cols * 4, rows * 4))  # Adjust the figure size based on the grid

for i, image_file in enumerate(image_files):
    img_path = os.path.join(output_directory, image_file)
    img = resize_image(img_path)

    plt.subplot(rows, cols, i + 1)
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Image {i + 1}")

plt.tight_layout()
plt.show()
